In [9]:
# KP 데이터
data_2020 = pd.read_csv('D:/STU/2학기/공모전/치안공모전/data/KP2020.csv')
data_2021 = pd.read_csv('D:/STU/2학기/공모전/치안공모전/data/KP2021.csv')

In [10]:
# 데이터 병합(2020,2021)
mdata = pd.concat([data_2020,data_2021])

In [13]:
# null값 제거를 위해 null값에 임시로 값 0으로 채워줌
mdata[['RPTER_SEX']] = mdata[['RPTER_SEX']].fillna(0)
mdata = mdata[mdata.RPTER_SEX != 0] # 0이 아닌 애들민 데이터프레임으로 만들기
mdata[['RPTER_SEX']] = mdata[['RPTER_SEX']].astype(int) # 나머지 애들 int로 변경
data_sex = pd.get_dummies(mdata['RPTER_SEX']) # 원핫인코딩. pd_get_dummides() 사용 
newdata = pd.concat([mdata,data_sex], axis=1) # 원핫 인코딩 된 df와 기존 df 합치기 = newdata로 생성
newdata = newdata.drop(['RPTER_SEX'],axis=1) # 기존 성별 컬럼 삭제

# 정답(215)을 1로, 나머지 값들은 0으로 만들기
def code(value):
    if value < 215:
        return 0
    elif value > 215:
        return 0
    elif value == 215:
        return 1

newdata['EVT_CL_CD'] = newdata['EVT_CL_CD'].apply(code) # 함수 적용
newdata['EVT_CL_CD'].value_counts() # 0과 1 개수 확인

0    2609335
1      35489
Name: EVT_CL_CD, dtype: int64

In [14]:
# datetime으로 변경하고 연월시간으로 나누기
data0 = newdata['RECV_CPLT_DM'] # 시리즈로 뽑아오기
data0 = data0.str.replace(r'.000000000',repl=r'', case=False, flags=0, regex=True) # 뒤애 0 9개 제거
data0 = data0.rename('RECV_CPLT_DM2') # 시리즈 이름 바꾸기
newdata = pd.concat([newdata, data0], axis=1) # 시리즈 기존 데이터프레임에 병합
newdata = newdata.drop(['RECV_CPLT_DM'],axis=1) # 기존 시간 컬럼 삭제
newdata = newdata.rename(columns={'RECV_CPLT_DM2':'RECV_CPLT_DM'}) # 열 이름 변경

# datetime으로 변경
newdata['RECV_CPLT_DM'] = newdata['RECV_CPLT_DM'].apply(lambda x: pd.to_datetime(str(x), format = '%y/%m/%d %H:%M:%S'))
newdata['year'] = newdata['RECV_CPLT_DM'].dt.year # 연 월 시간 새 컬럼으로 만들기
newdata['month'] = newdata['RECV_CPLT_DM'].dt.month
newdata['hour'] = newdata['RECV_CPLT_DM'].dt.hour

In [15]:
# 세종,대전,충남 그룹화
newdata = newdata.replace('세종로파출소', '세종')
newdata = newdata.replace('세종북부서', '세종')
newdata = newdata.replace('세종대왕파출소', '세종')
newdata = newdata.replace('세종남부서', '세종')
newdata = newdata.replace('세종서', '세종')
newdata = newdata.replace('세종청', '세종')
newdata = newdata.replace('충남청', '충남')
newdata = newdata.replace('2지구대(충남)', '충남')
newdata = newdata.replace('남대전지구대', '대전')
newdata = newdata.replace('대전대덕', '대전')
newdata = newdata.replace('대전대덕서', '대전')
newdata = newdata.replace('대전동부', '대전')
newdata = newdata.replace('대전동부서', '대전')
newdata = newdata.replace('대전둔산', '대전')
newdata = newdata.replace('대전둔산서', '대전')
newdata = newdata.replace('대전서부', '대전')
newdata = newdata.replace('대전서부서', '대전')
newdata = newdata.replace('대전역지구대', '대전')
newdata = newdata.replace('대전유성', '대전')
newdata = newdata.replace('대전유성서', '대전')
newdata = newdata.replace('서대전지구대', '대전')
newdata = newdata.replace('대전파출소', '대전')
newdata = newdata.replace('대전청', '대전')
newdata = newdata.replace('대전중부서', '대전')
newdata = newdata.replace('대전중부', '대전')

In [16]:
# 문자형 원핫 인코딩
lo_data = pd.get_dummies(newdata['RECV_DEPT_NM']) # 위에서 그룹화 한 지역명 원핫 인코딩
newdata = pd.concat([newdata,lo_data], axis=1) # 원핫 인코딩 된 df와 기존 df 합치기
newdata = newdata.drop(['RECV_DEPT_NM'],axis=1) # 기존 지역명 컬럼 삭제

# 그 외 불필요한 컬럼 삭제
newdata = newdata.drop(['HPPN_Y'],axis=1)
newdata = newdata.drop(['HPPN_X'],axis=1)

In [17]:
# null값 0으로 변경
newdata['HPPN_PNU_ADDR'] = newdata['HPPN_PNU_ADDR'].fillna(0)
newdata['SME_EVT_YN'] = newdata['SME_EVT_YN'].fillna(0)

In [18]:
newdata 

,NPA_CL,EVT_STAT_CD,EVT_CL_CD,HPPN_PNU_ADDR,SME_EVT_YN,1,2,3,RECV_CPLT_DM,year,month,hour,대전,세종,충남
0,19,10,0,충청남도 보령시 궁촌동(행정:대천4동) 369,Y,1,0,0,2020-12-01 01:43:07,2020,12,1,0,0,1
1,13,10,0,0,0,0,0,1,2020-12-01 02:05:04,2020,12,2,1,0,0
2,13,10,0,0,0,1,0,0,2020-12-01 02:06:52,2020,12,2,1,0,0
3,19,10,0,충청남도 보령시 천북면 하만리 628-10,N,0,0,1,2020-12-01 02:37:25,2020,12,2,0,0,1
4,19,10,0,충청남도 천안시 서북구 성정동(행정:성정2동) 1259,0,0,1,0,2020-12-01 08:17:50,2020,12,8,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2594055,13,10,0,대전광역시 유성구 봉명동 (행정:원신흥동 ) 1016-7,0,1,0,0,2023-01-12 11:23:09,2023,1,11,1,0,0
2594056,19,10,0,충청남도 서산시 음암면 부장리(음암면 ) 560-4,0,0,1,0,2023-01-16 11:59:47,2023,1,11,0,0,1
2594057,19,10,0,0,0,0,0,1,2023-01-16 11:58:54,2023,1,11,0,0,1
2594058,19,10,0,0,0,0,0,1,2023-01-14 01:00:47,2023,1,1,0,0,1


### 전처리 끝